# Multi-agent Customer Support Automation

In [73]:
!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain langchain-cohere



In [74]:
# from crewai import Agent, Task, Crew
from crewai import Crew, Agent, Task, Process

In [75]:
from langchain_community.chat_models import ChatCohere
import os

os.environ["COHERE_API_KEY"] = "hUQNZ8BBOMfdSMOtD3Go9QvIsvhLfxI2V9GvOcu0"
llm = ChatCohere(cohere_api_key=os.getenv("COHERE_API_KEY"))


## Tools

In [76]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [13]:
# Only this page will be scrapped but if you change the settings, Ii canbe differents:
docs_scrape_tool = ScrapeWebsiteTool(
website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/")

# docs_scrape_tool = ScrapeWebsiteTool(
#     website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/",
#     follow_links=True,  # اگر این ویژگی در ابزار پشتیبانی شود
#     depth=2  # میزان عمق لینک‌هایی که باید دنبال شوند
# )


### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [41]:
support_agent = Agent(
    role="Senior Support Representative",
	  goal="Be the most friendly and helpful "
        "support representative in your team",
	  backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	  ),
	  allow_delegation=False,
	  verbose=True,
    llm=llm
)

In [42]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
  llm=llm
 )

# in this agent it is a choise to delegate teh work or not, allow_delegation=True, they call "QA Agents, the agent based on their role and the response given by their co worker, they decide"


## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [43]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	  tools=[docs_scrape_tool],
    agent=support_agent,
)

In [44]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


## Creating the Crew


###Memory with cohere

In [61]:
# from crewai import Crew, Agent, Task, Process

# my_crew = Crew(
#     agents=[support_agent, support_quality_assurance_agent],  # ایجنت‌ها
#     tasks=[inquiry_resolution , quality_assurance_review],               # تسک‌ها
#     process=Process.sequential,                      # پردازش ترتیبی
#     memory=True,                                     # فعال کردن حافظه
#     verbose=True,
#     embedder={
#         "provider": "cohere",
#         "config": {
#             "api_key": "hUQNZ8BBOMfdSMOtD3Go9QvIsvhLfxI2V9GvOcu0",  # کلید API
#             "model": "embed-english-light-v3.0"                     # مدل Embedding
#         }
#     }
# )


In [70]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  # memory=True
)

## Running the Crew


#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [62]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}


In [ ]:
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: I can assist Andrew Ng and DeepLearningAI with their query about adding memory to their Crew. I'll provide a step-by-step guide and some additional tips to ensure they can effectively manage their Crew's memory.

Final Answer: Hi Andrew! It's great to hear from you, and I'm here to help with your Crew setup. Adding memory to your Crew is a straightforward process and can greatly enhance the team's productivity. Here's a comprehensive guide to get you

In [72]:
from IPython.display import Markdown
Markdown(result)

## Final Response to Customer:

Hi Andrew,

I'm thrilled to help you expand your CrewAI team's capabilities by adding memory, and I've crafted a detailed guide to make this process a breeze!

**Step-by-Step Guide to Enhancing Your Crew's Memory:**

**1. Accessing Crew Settings:**
   - Begin by logging into your CrewAI account.
   - Locate the specific Crew you want to enhance and click on the settings icon, which is typically a gear symbol or a 'Settings' button.

**2. Locating the Memory Section:**
   - In the Crew settings, you'll discover a dedicated section for memory or knowledge base management.
   **Fun fact:** This is the powerhouse of your team's knowledge!

**3. Creating New Memory Entries:**
   - Click on the 'Add Memory' button (or similar) to begin crafting a new memory.
   - Fill in the following fields:
     - **Title:** A concise title to identify the memory.
     - **Content:** The main body of your memory, where you can add text, images, and documents to make it engaging and informative.
     - **Tags:** Tags are like labels that help categorize your memories, making them easily searchable.
     - **Source:** If you're referencing external resources, include the source here for credibility.

**4. Saving and Organizing Memories:**
   - Click 'Save' to store your memory in the knowledge base.
   - Feel free to create multiple memories to build a comprehensive knowledge repository.
   - Organize them into folders or categories for easy access and better knowledge management.

**5. Access and Sharing:**
   - Your team can access these memories through the 'Knowledge' or 'Memory' tab in their Crew interface.
   - Encourage them to use the search feature for quick information retrieval.
   - Foster a culture of collaboration by inviting everyone to contribute, ensuring a dynamic and diverse knowledge base.

**Tips for an Exceptional Memory System:**
- **Regular Updates:** Assign a dedicated team member to review and update memories, keeping the knowledge fresh and relevant.
- **Consistency Matters:** Establish a consistent formatting style to ensure a smooth and enjoyable reading experience for your team.
- **Training and Onboarding:** Provide training sessions to introduce new and existing team members to the memory system, empowering them to contribute effectively.
- **Encourage Collaboration:** Create an environment where knowledge sharing is celebrated, fostering a collaborative spirit within your team.

**Advanced Support:**
For more advanced techniques and troubleshooting, our comprehensive documentation is an excellent resource. Additionally, our support team is always ready to assist if you encounter any challenges.

I hope this guide makes the process crystal clear, Andrew! Please feel free to reach out if you have any further questions or feedback. Your satisfaction is our priority.

Happy CrewAI adventures!

Best regards,
[Your Name]
CrewAI Support Team